Nessa base temos características de imóveis de retornam o valor total a ser gasto mensalmente. Tentarie prever qual o rent amount de acordo com os outras características.
#### Não faz sentido tentar prever o total sendo que ele é diretamente atralado ao rent amount.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import re

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn import metrics

import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/kaggle/input/brasilian-houses-to-rent/houses_to_rent.csv')

In [ ]:
df.head()

## Adequação dos dados

#print(*df.hoa.unique(), sep='\n')

#print(*df['property tax'].unique(), sep='\n')

Temos nas colunas hoe e property tax os valores: Incluso e Sem info. Vamos substituí-los por zero

In [ ]:
df['hoa'] = df['hoa'].replace('Sem info','0')
df['hoa'] = df['hoa'].replace('Incluso','0')

In [ ]:
df['property tax'] = df['property tax'].replace('Sem info','0')
df['property tax'] = df['property tax'].replace('Incluso','0')

In [ ]:
df.info()

Precisamos remover os símbolos R$ 

In [ ]:
df.total = df['total'].map(lambda x: re.sub('\D', '', x))
df.hoa = df.hoa.map(lambda x: re.sub('\D', '', x))
df['rent amount'] = df['rent amount'].map(lambda x: re.sub('\D', '', x))
df['property tax'] = df['property tax'].map(lambda x: re.sub('\D', '', x))
df['fire insurance'] = df['fire insurance'].map(lambda x: re.sub('\D', '', x))

# [.map()] Used for substituting each value in a Series with another value
# [\D] Matches any character which is not a decimal digit.

df.head(1)

In [ ]:
df.isnull().sum()

Transformar o tipo de object para float

In [ ]:
df.total = pd.to_numeric(df.total).astype(float)
df.hoa = pd.to_numeric(df.hoa).astype(float)
df['rent amount'] = pd.to_numeric(df['rent amount']).astype(float)
df['property tax'] = pd.to_numeric(df['property tax']).astype(float)
df['fire insurance'] = pd.to_numeric(df['fire insurance']).astype(float)

In [ ]:
df.info()

# Outliers

In [ ]:
plt.boxplot(df.total)

In [ ]:
df[df.total > 50000]

* O 1269 tem um valor absurdo de condomínio, 220000,00. O 3303 tem condomínio de 200000. Não deve estar correto
* O 2611 tem o valor de IPUT de 313700,00 com 42 metros quadrados. 
* O 5627 tem IPUT de 366300,00
* O único com valor plausível é o 3381. 

In [ ]:
df = df.drop([1269, 3303, 2611, 5627])

## Análise exploratória

In [ ]:
df.drop(columns=['total'], inplace=True)

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x='rooms', y='rent amount', data=df)

In [ ]:
plt.figure(figsize=(10,6))
sns.boxplot(x='bathroom', y='rent amount', data=df)

In [ ]:
plt.figure(figsize=(14,10))
minha_ordem = df.groupby('floor')['rent amount'].median().sort_values().iloc[::-1].index
sns.boxplot(x='floor', y='rent amount', data=df, order=minha_ordem)

Andares mais altos tem em geral custos maiores. 
O traço, que representa casas, está entre os valores maiores de 10. Vou substituí- lo por 10.

In [ ]:
df.floor = df.floor.map(lambda x: re.sub('-', '10', x))
df.floor = pd.to_numeric(df.floor).astype(float)

In [ ]:
df['animal'].value_counts().plot(kind='pie', autopct = '%.2f%%')

In [ ]:
df['furniture'].value_counts().plot(kind='pie', autopct = '%.2f%%')

In [ ]:
print(pd.DataFrame(df.groupby('furniture')['rent amount'].mean()))
df.groupby('furniture')['rent amount'].mean().plot(kind='barh')

In [ ]:
sns.pairplot(df, y_vars=['rent amount'], x_vars=['floor', 'hoa','property tax', 'fire insurance'])

O rent amount tem um relação forte com o fire insurance

In [ ]:
df_s = pd.DataFrame(preprocessing.scale(df[['area', 'rooms', 'bathroom', 'parking spaces', 'floor', 'hoa', 'rent amount',
                             'property tax', 'fire insurance']]))

In [ ]:
df[['area', 'rooms', 'bathroom', 'parking spaces', 'floor', 'hoa', 'rent amount', 'property tax', 'fire insurance']] = df_s

In [ ]:
df.isna().sum()

In [ ]:
df[df.isna().any(axis=1)]

As últimas quatro linhas são as que contem Nan

In [ ]:
df.drop(index=[6076, 6077, 6078, 6079],inplace=True)

In [ ]:
df.drop(columns=['Unnamed: 0'], inplace=True)
# Essa coluna não nos diz nada

## Seleção de variáveis

In [ ]:
df = pd.get_dummies(df, columns=['animal', 'furniture'], drop_first=True)

In [ ]:
df.head(1)

In [ ]:
X = df.drop(columns=['rent amount'])
y = df['rent amount']

In [ ]:
results = sm.OLS(y, X).fit()
print(results.summary())

O P-valor de animal acept é maior que 0,05 tem o maior valor. É o primeiro a ser retirado

In [ ]:
X = X.drop(columns=['animal_not acept'])
results = sm.OLS(y, X).fit()
print(results.summary())

O P-valor de area é maior que 0,05 tem o maior valor

In [ ]:
X = X.drop(columns=['area'])
results = sm.OLS(y, X).fit()
print(results.summary())

O P-valor de bathroom é maior que 0,05 tem o maior valor

In [ ]:
X = X.drop(columns=['bathroom'])
results = sm.OLS(y, X).fit()
print(results.summary())

O P-valor de city é maior que 0,05 tem o maior valor

In [ ]:
X = X.drop(columns=['city'])
results = sm.OLS(y, X).fit()
print(results.summary())

O P-valor de furniture_not furnished é maior que 0,05 tem o maior valor

In [ ]:
X = X.drop(columns=['furniture_not furnished'])
results = sm.OLS(y, X).fit()
print(results.summary())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=34)

## Modelagem e avaliação

In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
plt.scatter(y_test,predictions)

In [ ]:
score = r2_score(y_test, predictions)
print(score)

O valor máximo positivo do r2 é 1. Quanto mais próximo de 1 melhor

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))